In [14]:
from tes.data import Data,PacketStream,Packet,Payload,EventFlags
from tes.hdf5 import _PacketData, _import
from tes.base import Detection, Height, Timing
import numpy as np
import matplotlib.pyplot as plt
from subprocess import Popen

import logging
import sys
logger = logging.getLogger('tes')
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter('%(name)s:%(levelname)s %(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

trace_dt=np.dtype([('raw','i4'), ('f','i4'), ('s','i4')])
flags_dt = (np.int32, {'f0':(np.uint8,0),'f1':(np.uint8,1),'f2':(np.uint8,2), 'f3':(np.uint8,3)})

min_dt=np.dtype([
    ('clk','i4'),('f','i4'),('s','i4'),('low','i4'),('high','i4'),('flags',flags_dt),
    ('max_slope','i4'),('minima','i4'),('s_area','i4'),('s_extrema','i4')
])
max_dt=np.dtype([
    ('clk','i4'),('f','i4'),('s','i4'),('flags',flags_dt),('s_area','i4'),('s_extrema','i4'),
    ('peak_height','i4')
])
f0_xing_dt=np.dtype([
    ('clk','i4'),('f_area','i4'),('f_exrema','i4')
])
ptn_xing_dt=np.dtype([
    ('clk','i4'),('pulse_area','i4'),('pulse_length','i4'),('pulse_timer','i4')
])
rise_start_dt=np.dtype([
    ('clk','i4'),('pulse_timer','i4'),('rise_number','i4'),('rise_address','i4')
])
rise_dt=np.dtype([
    ('clk','i4'),('pulse_timer','i4'),('rise_number','i4'),('rise_address','i4'),
    ('rise_timer','i4'),('peak_height','i4')
])
pulse_start_dt=np.dtype([
    ('clk','i4'),('enabled','i4'),('area_threshold','i4'),('cfd_rel2min','i4'),
    ('constant_fraction','i4'), ('detection','i4'),('height','i4'),('max_peaks','i4'),
    ('pulse_threshold','i4'),('slope_threshold','i4'),('timing','i4')
])
stamp_dt=np.dtype([('clk','i4'),('pulse_timer','i4')])
f0_dt=np.dtype([
    ('rise0','u1'),('rise_start','u1'),('pulse_start','u1'),('will_cross','u1'),('will_arm','u1'),
    ('cfd_error','u1'),('cfd_overrun','u1'),('cfd_valid','u1')
])
f1_dt=np.dtype([
    ('b0','u1'),('b1','u1'),('b2','u1'),('b3','u1'),('b4','u1'),
    ('rise2','u1'),('rise1','u1'),('valid_rise','u1')
])

xing_dt=np.dtype([('clk','i4')])
stream_dt=np.dtype([('clk','i4'),('data','u8')])

def xings(s, thresh):
    xing = np.zeros_like(s)
    i=0
    was_above = s[0] > thresh
    for i in range(1, len(s)):
        if s[i] > thresh:
            if not was_above:
                xing[i] = 1
            was_above = True
        if s[i] < thresh:
            if was_above:
                xing[i] = -1
            was_above = False
            
    return np.where(xing==1)[0], np.where(xing==-1)[0]

In [15]:
traces=Data.fromfile('traces',trace_dt,'teslib','measurement_framer_TB',tool='vivado')
min_data=Data.fromfile('min_data',min_dt,'teslib','measurement_framer_TB',tool='vivado')
max_data=Data.fromfile('max_data',max_dt,'teslib','measurement_framer_TB',tool='vivado')
f0p_xings=Data.fromfile('f0p_xings',f0_xing_dt,'teslib','measurement_framer_TB',tool='vivado')
f0n_xings=Data.fromfile('f0n_xings',f0_xing_dt,'teslib','measurement_framer_TB',tool='vivado')
ptn_xings=Data.fromfile('ptn_xings',ptn_xing_dt,'teslib','measurement_framer_TB',tool='vivado')
rise_starts=Data.fromfile('rise_start',rise_start_dt,'teslib','measurement_framer_TB',tool='vivado')
rise_stops=Data.fromfile('rise_stop',rise_dt,'teslib','measurement_framer_TB',tool='vivado')
height_valids=Data.fromfile('height_valid',rise_dt,'teslib','measurement_framer_TB',tool='vivado')
init_reg=Data.fromfile('init_reg',pulse_start_dt,'teslib','measurement_framer_TB',tool='vivado')
pulse_starts=Data.fromfile('pulse_start',pulse_start_dt,'teslib','measurement_framer_TB',tool='vivado')
stamp_rises=Data.fromfile('stamp_rise',stamp_dt,'teslib','measurement_framer_TB',tool='vivado')
stamp_pulses=Data.fromfile('stamp_pulse',stamp_dt,'teslib','measurement_framer_TB',tool='vivado')
framer_errors=Data.fromfile('framer_error',xing_dt,'teslib','measurement_framer_TB',tool='vivado')
framer_overflows=Data.fromfile('framer_overflows',xing_dt,'teslib','measurement_framer_TB',tool='vivado')
commits=Data.fromfile('commit',xing_dt,'teslib','measurement_framer_TB',tool='vivado')
dumps=Data.fromfile('dump',xing_dt,'teslib','measurement_framer_TB',tool='vivado')
starts=Data.fromfile('start',xing_dt,'teslib','measurement_framer_TB',tool='vivado')
stream=Data.fromfile('stream',stream_dt,'teslib','measurement_framer_TB',tool='vivado')
raw=traces['raw']
f=traces['f']
s=traces['s']
mins, maxs = xings(s, 0)
f0p, f0n = xings(f,0)

In [16]:

if maxs[0] < mins[0]:
    maxs = maxs[1:] #start with a min
#     max_data=max_data[1:]
if f0n[0] < f0p[0]:
    f0n = f0n[1:]

#get initial registers
reg = init_reg[0]

# flags
min_f0=np.unpackbits(min_data['flags']['f0']).view(f0_dt)
min_f1=np.unpackbits(min_data['flags']['f1']).view(f1_dt)
max_f0=np.unpackbits(max_data['flags']['f0']).view(f0_dt)
max_f1=np.unpackbits(max_data['flags']['f1']).view(f1_dt)

# iterate over minima found by looking at traces
for i in range(len(mins)):
    if mins[i] != min_data['clk'][i]:
        print('index mismatch for minima i={}'.format(i))
        break
    if maxs[i] != max_data['clk'][i]:
        print('index mismatch for maxima i={}'.format(i))
        break
        
    #look for a pulse start at same clk index as this min
    ps = np.where(pulse_starts['clk'][i]==mins[i])[0]
    if len(ps):
        reg = pulse_starts[ps[0]]
        
    maxima = f[maxs[i]]
    minima = f[mins[i]]
    will_cross = maxima > reg['pulse_threshold']
    will_arm = len(np.where(s[mins[i]:maxs[i]+1] > reg['slope_threshold']))!=0
    first_rise = minima <= reg['pulse_threshold']
    
    if will_cross != (min_f0['will_cross'][i]==1):
        print('will_cross mismatch @ min i={} '.format(i))
    if will_cross != (max_f0['will_cross'][i]==1):
        print('will_cross mismatch @ max i={}'.format(i))
        
    if will_arm != (min_f0['will_arm'][i]==1):
        print('will_arm mismatch @ min i={} '.format(i))
    if will_arm != (max_f0['will_arm'][i]==1):
        print('will_arm mismatch @ max i={}'.format(i))
    
    if first_rise != (min_f0['rise0'][i]==1):
        print('first_rist mismatch @ min i={}'.format(i))
    if first_rise != (max_f0['rise0'][i]==1):
        print('first_rise mismatch @ max i={}'.format(i))
    
    if first_rise:
        cfi = maxima*reg['constant_fraction']
        cf_min = 0
    else:
        cfi = (maxima-minima)*reg['constant_fraction']
        cf_min = minima
        
    cfi_f,_ = np.modf(cfi)
    cfi_i = np.right_shift(cfi,17)
    cfd_high = maxima - cfi_i
    cfd_low = cf_min + cfi_i

    if min_data['high'][i] != cfd_high:
        print('cfd_high mismatch i={} min_data={} cfd_high={}'.format(i,min_data['high'][i],cfd_high))
    if min_data['low'][i] != cfd_low:
        print('cfd_low mismatch i={} min_data={} cfd_high={}'.format(i,min_data['low'][i],cfd_low))
    cfd_error = cfd_low < minima
    if cfd_error != (min_f0['cfd_error'][i]=='1'):
        print('cfd_error mismatch i={}'.format(i))
    if cfd_error != (min_f0['cfd_valid'][i]=='0'):
        print('cfd_valid mismatch @ min i={}'.format(i))
    if cfd_error != (max_f0['cfd_valid'][i]=='0'):
        print('cfd_valid mismatch @ max i={}'.format(i))
    
    detection=Detection(reg['detection'])
    timing=Timing(reg['timing'])
    height=Height(reg['height'])
    if timing==Timing.cfd_low or height==Height.cfd_high or height==Height.cfd_height:
        valid_rise = will_arm and will_cross and cfd_valid
    else:
        valid_rise = will_arm and will_cross
    if valid_rise != min_f1['valid_rise'][i]:
        print('valid_rise mismatch @ min i={}'.format(i))
    if valid_rise != max_f1['valid_rise'][i]:
        print('valid_rise mismatch @ max i={}'.format(i))
    
    if valid_rise != min_f0['rise_start'][i]==1:
        print('rise_start missing @ min i={}'.format(i))
    if (valid_rise and first_rise) != min_f0['pulse_start'][i]==1:
        print('pulse_start missing @ min i={}'.format(i))
    
       

IndexError: index 1157 is out of bounds for axis 0 with size 1157

In [ ]:
j=2
mins[j],min_data['clk'][j],maxs[j],max_data['clk'][j]

In [ ]:
init_reg['detection'][0]

In [ ]:
min_f0['will_cross']

In [ ]:
min_data['f'][i],f[mins[i]:mins[i]+10]

In [ ]:
min_data['s'][i]

In [ ]:
min_data['clk'][i],mins[i]

In [ ]:
%matplotlib notebook
for k in range(1, 1000):
#     plt.plot([0,l],[0,0],':k')
#     plt.plot(s[mins[k]:maxs[k]+1],'r')
    plt.plot(f[mins[k]:maxs[k]+1],'b')